In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')
import gc

import datetime
# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import math

import textwrap
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
import pydotplus
from IPython.display import Image
from graphviz import Digraph
from sklearn import tree


pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 100)


In [2]:
df = pd.read_csv("../data/viewing_rating.csv", encoding='cp932')

In [3]:
# time型に
df.start_at=pd.to_datetime(df.start_at)
# averageの％を削除
df["average"] =  df["average"].apply(lambda x : float(x[:-1]) )


In [4]:
df["start_at_year"] = df["start_at"].dt.year
df["start_at_month"] = df["start_at"].dt.month

In [5]:
df["drama_key"] = df.start_at.dt.strftime("%y")+df.start_at.dt.strftime("%m")+"_"+df.TV_station+"_"+df.time_table

In [6]:
import json

f = open("../data/drama_info0115.json", 'r')
print(f)
drama_info_json = json.load(f)

<_io.TextIOWrapper name='../data/drama_info0115.json' mode='r' encoding='UTF-8'>


In [7]:
df_pre = pd.read_csv("../data/drama_pre_stage.csv", encoding='cp932')

In [8]:
df_pre = df_pre[["drama_key" ,"pre_part", "part_count"]]


In [9]:
print (textwrap.shorten(str(drama_info_json), 1000))

{'1910_CX_月21': {'main_actor1': 'ディーン・フジオカ', 'main_actor2': '岩田剛典', 'original_work': 'アーサー・コナン・ドイル「シャーロック・ホームズ」'}, '1910_CX_火22': {'main_actor1': '阿部寛', 'main_actor2': '吉田羊', 'original_work': ''}, '1910_TBS_火22': {'main_actor1': '波瑠', 'main_actor2': '中川大志', 'original_work': 'いくえみ綾「G線上のあなたと私」'}, '1910_NTV_水22': {'main_actor1': '高畑充希', 'main_actor2': '橋本愛', 'original_work': ''}, '1910_EX_木21': {'main_actor1': '米倉涼子', 'main_actor2': 'ユースケ・サンタマリア', 'original_work': ''}, '1910_CX_木22': {'main_actor1': '新木優子', 'main_actor2': '高良健吾', 'original_work': '瀧波ユカリ「モトカレマニア」'}, '1910_NTV_木23': {'main_actor1': '本田翼', 'main_actor2': '金子大地', 'original_work': ''}, '1910_TBS_金22': {'main_actor1': '福士蒼汰', 'main_actor2': '菜々緒', 'original_work': 'キリエ「4分間のマリーゴールド」'}, '1910_EX_金23': {'main_actor1': 'オダギリジョー', 'main_actor2': '麻生久美子', 'original_work': ''}, '1910_NTV_土22': {'main_actor1': '生田斗真', 'main_actor2': '安田顕', 'original_work': ''}, '1910_EX_土23': {'main_actor1': '田中圭', 'main_actor2': '吉田鋼太郎', [...]


In [10]:
df["main_actor1"] = ""
df["main_actor2"] = ""
df["original_work"] = ""
df["pre_part"] = 0.0
df["part_count"] = 0

In [11]:
for tmp_index in range(len(df)):
    tmp_drama_key = df.iloc[tmp_index]["drama_key"]
    if tmp_drama_key in drama_info_json.keys():
#         print(drama_info_json[tmp_drama_key]["main_actor1"])
        df["main_actor1"][tmp_index] = drama_info_json[tmp_drama_key]["main_actor1"]
        df["main_actor2"][tmp_index] = drama_info_json[tmp_drama_key]["main_actor2"]
        df["original_work"][tmp_index] = drama_info_json[tmp_drama_key]["original_work"]
        
        if tmp_drama_key in df_pre.drama_key.values:
            df["pre_part"][tmp_index] = \
                df_pre[df_pre["drama_key"] == tmp_drama_key]["pre_part"].values[0]
            df["part_count"][tmp_index] = \
                df_pre[df_pre["drama_key"] == tmp_drama_key]["part_count"].values[0]
#             float(df_pre[df_pre["drama_key"] == tmp_drama_key]["pre_part"].values[0])
#         else:
#             print(tmp_drama_key)
    
    #         print("yes"+tmp_drama_title)
    else:
        print(tmp_drama_key)

In [12]:
def make_person_key_df (actor_actress):
    actor_actress_list = []
    for column in actor_actress.columns:
        if column == "ranking":
            continue
        actor_actress_list.extend(actor_actress[column].values.tolist())
    print ("元の数"+ str(len(actor_actress_list)))
    # uniqueにする
    actor_actress_list = list(set(actor_actress_list))
    #nan を除去
    actor_actress_list = [actor for actor in actor_actress_list if type(actor) is not float ]
    print ("後の数"+str(len(actor_actress_list)))

    df_pivot = pd.DataFrame( columns=actor_actress_list, index=[str(n) for n in range(2008,2020)])

    #pivot 形式に整形する
    for tmp_name in df_pivot.columns:
        for year in actor_actress.columns:
            if year == "ranking": continue
            if  len(actor_actress[actor_actress[year] == tmp_name].ranking) > 0:
                df_pivot.loc[year][tmp_name] = (actor_actress[actor_actress[year] == tmp_name].ranking).values[0]
    return (df_pivot)
    

In [13]:
df_actress = pd.read_csv("../data/actress_power.csv", encoding='cp932')
df_actress_pivot = make_person_key_df(df_actress)


元の数360
後の数69


In [14]:
df_actor = pd.read_csv("../data/actor_power.csv", encoding='cp932')
df_actor_pivot = make_person_key_df(df_actor)

元の数372
後の数69


In [15]:
df_actress_pivot

,竹内結子,米倉涼子,桐谷美玲,山口智子,柴咲コウ,橋本環奈,江角マキコ,綾瀬はるか,上戸,波瑠,柴崎コウ,北川景子,杏,菜々緒,黒木瞳,上野樹里,深津絵里,宮崎あおい,上戸彩,松たか子,石原さとみ,天海祐希,仲間由紀恵,多部未華子,長澤まさみ,真矢みき,武井咲,堀北真希,吉永小百合,広瀬すず,優香,観月ありさ,松嶋菜々子,小泉今日子,蒼井優,松,榮倉奈々,小雪,芦田愛菜,樹木希林,菅野美穂,松雪泰子,深田恭子,佐々木希,黒木メイサ,石田ゆり子,松下奈緒,有村架純,香里奈,新垣結衣,井上真央,戸田恵梨香,常盤貴子,藤原紀香,木村文乃,篠原涼子,吉瀬美智子,能年玲奈,吉高由里子,土屋太鳳,薬師丸ひろ子,高畑充希,宮_あおい,永作博美,吉田羊,広瀬アリス,剛力彩芽,木村佳乃,武井
2008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010,8,19,NaN,11,7,NaN,NaN,6,NaN,NaN,NaN,NaN,NaN,NaN,14,NaN,13,NaN,4,16,NaN,1,2,NaN,17,18,NaN,15,22,NaN,12,29,3,NaN,26,NaN,NaN,30,NaN,NaN,5,NaN,20,NaN,23,NaN,NaN,NaN,NaN,9,28,25,24,27,NaN,21,NaN,NaN,NaN,NaN,NaN,NaN,10,NaN,NaN,NaN,NaN,NaN,NaN
2011,13,26,NaN,28,4,NaN,NaN,2,NaN,NaN,NaN,15,NaN,NaN,30,18,11,NaN,3,19,21,6,5,NaN,NaN,20,NaN,14,22,NaN,NaN,24,8,16,25,NaN,NaN,NaN,NaN,NaN,1,NaN,17,NaN,7,NaN,23,NaN,NaN,10,27,NaN,NaN,NaN,NaN,12,NaN,NaN,NaN,NaN,NaN,NaN,9,29,NaN,NaN,NaN,NaN,NaN
2012,8,26,NaN,NaN,10,NaN,17,2,6,NaN,NaN,18,NaN,NaN,NaN,NaN,11,16,NaN,NaN,23,3,4,NaN,12,27,NaN,5,29,NaN,14,NaN,1,22,NaN,24,NaN,NaN,20,NaN,9,30,21,NaN,NaN,NaN,NaN,NaN,25,7,13,NaN,28,NaN,NaN,15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19
2013,25,11,NaN,NaN,8,NaN,15,1,NaN,NaN,NaN,14,NaN,NaN,NaN,NaN,16,24,4,NaN,19,3,7,NaN,17,22,26,2,27,NaN,12,NaN,5,21,NaN,NaN,NaN,NaN,NaN,18,6,NaN,13,NaN,NaN,NaN,NaN,NaN,NaN,9,20,NaN,23,NaN,NaN,10,NaN,NaN,28,NaN,29,NaN,NaN,NaN,NaN,NaN,30,NaN,NaN
2014,NaN,13,NaN,NaN,7,NaN,20,1,NaN,NaN,NaN,4,15,NaN,NaN,NaN,27,NaN,3,23,8,9,10,NaN,22,NaN,21,2,19,NaN,16,26,12,11,NaN,NaN,NaN,NaN,NaN,24,6,NaN,18,NaN,NaN,NaN,29,NaN,NaN,5,17,NaN,NaN,NaN,NaN,14,NaN,25,NaN,NaN,28,NaN,30,NaN,NaN,NaN,NaN,NaN,NaN
2015,16,9,29,NaN,6,NaN,NaN,1,NaN,NaN,NaN,4,7,NaN,NaN,30,28,NaN,10,11,12,8,3,NaN,22,NaN,23,2,25,NaN,19,NaN,14,24,NaN,NaN,NaN,NaN,NaN,NaN,15,NaN,18,21,NaN,NaN,NaN,NaN,NaN,5,20,NaN,NaN,NaN,NaN,13,27,17,26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016,26,20,10,NaN,13,NaN,NaN,1,NaN,NaN,NaN,2,11,27,NaN,NaN,28,NaN,12,17,5,7,9,NaN,22,NaN,23,6,25,15,24,NaN,21,19,NaN,NaN,NaN,NaN,NaN,NaN,18,NaN,8,NaN,NaN,NaN,NaN,3,NaN,4,30,NaN,NaN,NaN,NaN,14,NaN,NaN,NaN,NaN,NaN,NaN,29,NaN,16,NaN,NaN,NaN,NaN
2017,29,18,9,NaN,NaN,NaN,NaN,2,NaN,NaN,13,4,14,NaN,NaN,NaN,25,NaN,12,NaN,3,8,7,24,16,NaN,23,15,NaN,11,NaN,NaN,22,27,NaN,NaN,26,NaN,NaN,NaN,17,NaN,5,30,NaN,20,NaN,6,NaN,1,28,NaN,NaN,NaN,NaN,10,NaN,NaN,21,19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
df_actor_namekey = df_actor_pivot.T.rename_axis('actor_actress').reset_index()

In [17]:
df_actress_namekey = df_actress_pivot.T.rename_axis('actor_actress').reset_index()

In [18]:
df_actress_namekey

,actor_actress,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,竹内結子,NaN,NaN,8,13,8,25,NaN,16,26,29,NaN,21
1,米倉涼子,NaN,NaN,19,26,26,11,13,9,20,18,15,12
2,桐谷美玲,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29,10,9,20,23
3,山口智子,NaN,NaN,11,28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,柴咲コウ,NaN,NaN,7,4,10,8,7,6,13,NaN,16,15
5,橋本環奈,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18
6,江角マキコ,NaN,NaN,NaN,NaN,17,15,20,NaN,NaN,NaN,NaN,NaN
7,綾瀬はるか,NaN,NaN,6,2,2,1,1,1,1,2,1,1
8,上戸,NaN,NaN,NaN,NaN,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,波瑠,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26,25


In [19]:
df_merged = pd.merge(df, df_actor_namekey,left_on='main_actor1', right_on='actor_actress', how='left')
#df_merged = pd.merge(df_merged, df_actress_namekey,left_on='main_actor1', right_on='actor_actress', how='right')
#df_merged = pd.merge(df_merged, df_actress_namekey,left_on='main_actor2', right_on='actor_actress', how='right')

In [20]:

df_merged = pd.merge(df_merged, df_actress_namekey,left_on='main_actor1', right_on='actor_actress', how='left'
                     ,suffixes=('_1', '_2'))

In [21]:

df_merged = pd.merge(df_merged, df_actor_namekey,left_on='main_actor2', right_on='actor_actress', how='left'
                     ,suffixes=('_3', '_4') )


In [22]:
df_merged = pd.merge(df_merged, df_actress_namekey,left_on='main_actor2', right_on='actor_actress', how='left'
                    , suffixes=('_3', '_4'),)

In [23]:
df_merged.loc[:,df_merged.columns.str.contains('20')]

,2008_1,2009_1,2010_1,2011_1,2012_1,2013_1,2014_1,2015_1,2016_1,2017_1,2018_1,2019_1,2008_2,2009_2,2010_2,2011_2,2012_2,2013_2,2014_2,2015_2,2016_2,2017_2,2018_2,2019_2,2008_3,2009_3,2010_3,2011_3,2012_3,2013_3,2014_3,2015_3,2016_3,2017_3,2018_3,2019_3,2008_4,2009_4,2010_4,2011_4,2012_4,2013_4,2014_4,2015_4,2016_4,2017_4,2018_4,2019_4
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,11,3,1,1,1,1,1,2,1,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16,NaN,NaN,27
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26,25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19,26,26,11,13,9,20,18,15,12,NaN,NaN,25,26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
937,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
938,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,8,1,5,12,14,21,22,18,19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,3,NaN,4,3,10,12,12,11,NaN
940,NaN,NaN,1,1,3,2,3,3,2,4,3,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
df_new8years = df_merged[df_merged["start_at"] > datetime.datetime.strptime("2011-01-01", '%Y-%m-%d')]

In [25]:
df_new8years[ df_new8years.pre_part>0].drama_title.sort_values

<bound method Series.sort_values of 1          まだ結婚できない男
4              ドクターX
8         時効警察はじめました
10           おっさんずラブ
30             緊急取調室
34          家政夫のミタゾノ
42         家売るオンナの逆襲
63            下町ロケット
65              絶対零度
86          家政夫のミタゾノ
102             99.9
108            ドクターX
116      コード・ブルー 3rd
120             遺留捜査
134            緊急取調室
160            ドクターX
217            コウノドリ
225        花咲舞が黙ってない
258         DOCTORS3
272            MOZU2
273     ママとパパが生きる理由。
274            ドクターX
282             HERO
285              GTO
288           信長のシェフ
300        なるようになるさ。
306          刑事110キロ
307             MOZU
309      続・最後から二番目の恋
314        ファースト・クラス
319        チーム・バチスタ4
324              医龍4
337           リーガルハイ
339            ドクターX
344           都市伝説の女
350          救命病棟24時
353         ショムニ2013
355         DOCTORS2
359        警部補　矢部謙三2
360            斉藤さん2
364             ガリレオ
367             遺留捜査
380          ハンチョウ 6
415             遺留捜査
425          ハンチョウ 5
471             絶対零

In [26]:
df_new8years["main_actor1_rank"] = np.nan
df_new8years["main_actor2_rank"] = np.nan



In [27]:
for i in range(len(df_new8years)):
    prev_year = df_new8years["start_at"].dt.year.iloc[i] - 1 
    df_filter = df_new8years.loc[:,df_new8years.columns.str.contains(str(prev_year))]
#   import pdb;pdb.set_trace()
    
    prev_year1 = str(prev_year) + "_1"
    prev_year2 = str(prev_year) + "_2"
    prev_year3 = str(prev_year) + "_3"
    prev_year4 = str(prev_year) + "_4"
#     import pdb; pdb.set_trace()
    for j in range(len(df_filter)):
        tmp1 =  0 if np.isnan(df_filter[prev_year1].iloc[j]) else df_filter[prev_year1].iloc[j]
        tmp2 =  0 if np.isnan(df_filter[prev_year2].iloc[j]) else df_filter[prev_year2].iloc[j]
        tmp3 =  0 if np.isnan(df_filter[prev_year3].iloc[j]) else df_filter[prev_year3].iloc[j]
        tmp4 =  0 if np.isnan(df_filter[prev_year4].iloc[j]) else df_filter[prev_year4].iloc[j]
#         import pdb; pdb.set_trace()
        df_new8years["main_actor1_rank"].iloc[j] =   tmp1 + tmp2 if (tmp1 + tmp2 != 0) else np.nan
        df_new8years["main_actor2_rank"].iloc[j] =   tmp3 + tmp4 if (tmp3 + tmp4 != 0) else np.nan
#         print(dbg)
#     df_filter[prev_year + "_2"]
#    if  np.isnan(df_filter["2018_1"]) or np.isnan(df_filter["2018_2"])  :
#        print("come")

In [28]:
# df_new8years["main_actor1_rank"]

In [29]:
# df_new8years["main_actor2_rank"]

In [30]:
df_episode = df_new8years.loc[:, df_new8years.columns.str.contains("Epi")]

In [31]:
df_new8years["average_calc"] = np.nan

In [32]:
# averageを計算してみたが、もともとあるaverageが加重平均だったので、そちらを採用することに
for i in  range(len(df_episode)):
    vr_sum = 0
    for epi_i, col in  enumerate(df_episode.columns):
        if df_episode[col].iloc[i] == "-"  :
            if epi_i == 0:
#                 print (col+"_"+str(epi_i))
                break
            df_new8years["average_calc"].iloc[i] = vr_sum/epi_i
            break
        vr_sum += float(df_episode[col].iloc[i])
        if epi_i == 14 : df_new8years["average_calc"].iloc[i] = vr_sum / 15
        
    

### 視聴率を枠ごとに正規化する

In [47]:
norm_json = {}

for tmp_time in df_new8years.time_table:
    tmp_list = []
    for tmp_average in df_new8years [df_new8years["time_table"] == tmp_time].average:
        tmp_list.append(tmp_average)
#         print(len(tmp_average))
        
    
    norm_json[tmp_time] = sum(tmp_list)/ len(tmp_list)
    print(sum(tmp_list)/ len(tmp_list))
    print(len(tmp_list))
    
# pd.pivot_table(df_new8years.time_table)
# hoge

11.849714285714283
35
9.046739130434784
46
9.046739130434784
46
10.930851063829785
47
11.622115384615382
52
9.010857142857143
35
3.650857142857142
35
9.251999999999997
35
6.9791428571428575
35
9.09090909090909
11
5.112631578947369
19
11.856938775510205
49
8.51578947368421
19
11.849714285714283
35
9.490714285714287
28
9.046739130434784
46
10.930851063829785
47
11.622115384615382
52
9.010857142857143
35
3.650857142857142
35
9.251999999999997
35
6.9791428571428575
35
9.09090909090909
11
5.112631578947369
19
11.856938775510205
49
8.51578947368421
19
11.849714285714283
35
9.490714285714287
28
9.046739130434784
46
10.930851063829785
47
11.622115384615382
52
9.010857142857143
35
3.650857142857142
35
9.251999999999997
35
6.9791428571428575
35
9.09090909090909
11
5.112631578947369
19
11.856938775510205
49
8.51578947368421
19
11.849714285714283
35
9.490714285714287
28
9.046739130434784
46
10.930851063829785
47
11.622115384615382
52
9.010857142857143
35
3.650857142857142
35
9.251999999999997
35
6

In [48]:
# norm_json

{'月21': 11.849714285714283,
 '火22': 9.046739130434784,
 '水22': 10.930851063829785,
 '木21': 11.622115384615382,
 '木22': 9.010857142857143,
 '木23': 3.650857142857142,
 '金22': 9.251999999999997,
 '金23': 6.9791428571428575,
 '土22': 9.09090909090909,
 '土23': 5.112631578947369,
 '日21': 11.856938775510205,
 '日22': 8.51578947368421,
 '火21': 9.490714285714287,
 '木20': 9.520000000000001,
 '土21': 10.372083333333334,
 '月20': 8.674285714285714,
 '水21': 11.1525,
 '金20': 5.890000000000001,
 '日23': 5.862500000000001}

In [51]:
df_new8years["slot_vr_average"] = df_new8years["time_table"].apply(lambda tt : norm_json[tt]  )

In [57]:
df_new8years["pre_part_upfactor"] = 0

In [58]:

for i in range(len(df_new8years[df_new8years.pre_part > 0])):
    tmp_pre_part = df_new8years[df_new8years.pre_part > 0].iloc[i]["pre_part"]
    tmp_slot_vr_av = df_new8years[df_new8years.pre_part > 0].iloc[i]["slot_vr_average"]
    if tmp_pre_part > tmp_slot_vr_av:
        df_new8years["pre_part_upfactor"] = tmp_pre_part - tmp_slot_vr_av
    
    

In [59]:
df_new8years[df_new8years["pre_part_upfactor"]> 0]

,TV_station,time_table,drama_title,Episode1,Episode2,Episode3,Episode4,Episode5,Episode6,Episode7,Episode8,Episode9,Episode10,Episode11,Episode12,Episode13,Episode14,Episode15,average,suii,start_at,start_at_year,start_at_month,drama_key,main_actor1,main_actor2,original_work,pre_part,part_count,actor_actress_1,2008_1,2009_1,2010_1,2011_1,2012_1,2013_1,2014_1,2015_1,2016_1,2017_1,2018_1,2019_1,actor_actress_2,2008_2,2009_2,2010_2,2011_2,2012_2,2013_2,2014_2,2015_2,2016_2,2017_2,2018_2,2019_2,actor_actress_3,2008_3,2009_3,2010_3,2011_3,2012_3,2013_3,2014_3,2015_3,2016_3,2017_3,2018_3,2019_3,actor_actress_4,2008_4,2009_4,2010_4,2011_4,2012_4,2013_4,2014_4,2015_4,2016_4,2017_4,2018_4,2019_4,main_actor1_rank,main_actor2_rank,average_calc,slot_vr_average,pre_part_upfactor
0,CX,月21,シャーロック,12.8,9.3,9.9,10.6,9.3,8.3,9.9,8.9,9.9,8.8,9.8,-,-,-,-,9.90,5-5-0,2019-10-01,2019,10,1910_CX_月21,ディーン・フジオカ,岩田剛典,アーサー・コナン・ドイル「シャーロック・ホームズ」,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.772727,11.849714,7.343061
1,CX,火22,まだ結婚できない男,11.5,7.7,10.0,9.5,10,8.9,8.5,8.6,9,9.7,-,-,-,-,-,9.40,5-4-0,2019-10-01,2019,10,1910_CX_火22,阿部寛,吉田羊,,16.9,2.0,阿部寛,NaN,NaN,11,3,1,1,1,1,1,2,1,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,吉田羊,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16,NaN,NaN,27,11.0,NaN,9.340000,9.046739,7.343061
2,TBS,火22,G線上のあなたと私,7.8,8.8,7.3,6.9,6.7,6.7,8.6,7.6,6.5,9.6,-,-,-,-,-,7.65,2003/5/1,2019-10-01,2019,10,1910_TBS_火22,波瑠,中川大志,いくえみ綾「G線上のあなたと私」,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,波瑠,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26,25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.650000,9.046739,7.343061
3,NTV,水22,同期のサクラ,8.1,9.5,9.3,11.5,11.8,11.7,12.2,10.8,10.3,13.7,-,-,-,-,-,10.89,5-4-0,2019-10-01,2019,10,1910_NTV_水22,高畑充希,橋本愛,,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,高畑充希,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.890000,10.930851,7.343061
4,EX,木21,ドクターX,20.3,19.0,18.1,17.8,15.9,19.3,18.6,17.4,18.2,19.3,-,-,-,-,-,18.48,3-6-0,2019-10-01,2019,10,1910_EX_木21,米倉涼子,ユースケ・サンタマリア,,20.9,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,米倉涼子,NaN,NaN,19,26,26,11,13,9,20,18,15,12,ユースケ・サンタマリア,NaN,NaN,25,26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.0,25.0,18.390000,11.622115,7.343061
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
493,EX,金23,犬を飼うということ,9.3,8.3,7.8,8.6,9.4,7.4,8.1,8.3,9,-,-,-,-,-,-,8.47,5-3-0,2011-04-01,2011,4,1104_EX_金23,錦戸亮,水川あさみ,,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.466667,6.979143,7.343061
494,NTV,土21,高校生レストラン,13.1,11.2,10.9,11.3,11.5,8.7,10.2,9.5,9.7,-,-,-,-,-,-,10.75,4-4-0,2011-04-01,2011,4,1104_NTV_土21,松岡昌宏,吹石一恵,村林新吾「高校生レストラン､本日も満席｡」「高校生レストラン､行列の理由｡」,0.0,NaN,松岡昌宏,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25,NaN,30,27,29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.677778,10.372083,7.343061
495,TBS,日21,JIN -仁-,23.7,18.4,20.4,20.7,20.8,18.8,18.8,18.7,19.2,21.2,26.1,-,-,-,-,21.28,2006/3/1,2011-04-01,2011,4,1104_TBS_日21,大沢たかお,綾瀬はるか,村上もとか「JIN -仁-」,19.2,2.0,NaN,NaN,NaN

## CM数も特徴量に

In [ ]:
df_cm = pd.read_csv("../data/number_of_cm.csv", encoding='cp932')

In [ ]:
df_cm["name_ns"] = df_cm.name.str.replace('\s','')

In [ ]:
df_cm_pivot = pd.DataFrame(index = set(df_cm.name_ns.values),columns = range(1990,2019)  )

In [ ]:
for t_name in set(df_cm.name_ns.values):
    for t_year in range(1990,2019):
        
#         tmp_q = f"name_ns == '{t_name}' & year == '{t_year}' " 
        tmp_q = "name_ns == @t_name & year == @t_year " 
# print(tmp_q)
        try :
            tmp_df = df_cm.query(tmp_q)
    #             print(f'{tmp_df["number_of_cm"].values} タレント{t_name},{t_year}年')
            if tmp_df["number_of_cm"].values:
#                 print(f'{tmp_df["number_of_cm"].values} タレント{t_name},{t_year}年')
                df_cm_pivot.loc[t_name,t_year] = int(tmp_df["number_of_cm"].values)
        except KeyError as instance:
            print (instance)
        
        

In [ ]:
df_cm_pivot = df_cm_pivot.rename_axis("actor_actress").reset_index()

In [ ]:
rank_arr = df_actor_namekey.actor_actress.values.tolist()

In [ ]:
rank_arr.extend(df_actress_namekey.actor_actress.values)

# 特徴量エンジニアリング

### part3以降のものにフラグ

In [ ]:
df_new8years["is_than_part3"] = 0

In [ ]:
for tmp_idx in range(len(df_new8years)):
    if df_new8years["part_count"].iloc[tmp_idx] > 2:
        df_new8years["is_than_part3"].iloc[tmp_idx] = 1
    
    

In [ ]:
df_new8years["is_than_part3"]

In [ ]:
df_new8years[df_new8years["is_than_part3"] == True]

### 前の枠の数字

In [ ]:
df_new8years["pre_slot_vr_average"] = np.nan

In [ ]:
val_for_nan = sum(df_new8years.average)/len(df_new8years)

In [ ]:
for tmp_idx in range(len(df_new8years)):
    if df_new8years["start_at_month"].iloc[tmp_idx] == 1:
        pre_slot_month = str(10) 
        pre_slot_year = df_new8years["start_at_year"].iloc[tmp_idx] - 1
    else:
        pre_slot_month = "0"+ str(int(df_new8years["start_at_month"].iloc[tmp_idx]) - 3) 
        pre_slot_year = df_new8years["start_at_year"].iloc[tmp_idx]
        
    time_table = df_new8years["time_table"].iloc[tmp_idx]
    TV_station = df_new8years["TV_station"].iloc[tmp_idx]
#     print(f"{pre_slot_year%2000}{pre_slot_month}_{time_table}_{TV_station} ")
    pre_key =  str(pre_slot_year%2000) + pre_slot_month+"_"+TV_station+"_"+time_table
    try:
        if pre_key in df["drama_key"].values:
            tmp_pre_slot_vr_average = float(df[df["drama_key"]==pre_key].average) 
        else:
            tmp_pre_slot_vr_average = val_for_nan
#         print(pre_key)
    except:
        print(pre_key)
        tmp_pre_slot_vr_average = float(df[df["drama_key"]==pre_key].average.values[0])
    
    print(tmp_pre_slot_vr_average)
    
    df_new8years["pre_slot_vr_average"].iloc[tmp_idx] = tmp_pre_slot_vr_average
#         float(df[df["drama_key"]==pre_key].average)

In [ ]:
df_new8years = df_new8years.fillna({'part_count': 0})

### 俳優ランクを人気ポイントに

In [ ]:
df_new8years["main_actor1_pt"] = df_new8years["main_actor1_rank"].apply(lambda x : (41 - x)/40 )

In [ ]:
df_new8years["main_actor2_pt"] = df_new8years["main_actor2_rank"].apply(lambda x : (41 - x)/40 )

In [ ]:
df_new8years["main_actor1_pt"] = df_new8years["main_actor1_pt"].fillna(0)
df_new8years["main_actor2_pt"] = df_new8years["main_actor2_pt"].fillna(0)

In [ ]:
df_new8years["has_original_work"] = df_new8years["original_work"].apply(lambda x : 0 if x =="" else 1 ) 

### 年を特徴量エンジニアリング

In [ ]:
df_new8years["year_feature"] =  df_new8years["start_at_year"] - 2010

### (TODO) CM数も特徴量に加工する

### one hotエンコーディング

In [ ]:
# df_new8years = pd.get_dummies(df_new8years, columns=["TV_station", "start_at_month", "start_at_year","time_table"] )
df_new8years = pd.get_dummies(df_new8years, columns=["TV_station", "start_at_month","time_table"] )

## wikiの俳優情報を特徴量として

In [ ]:
f = open("../data/actor_wiki0220.json", 'r')
print(f)
actor_wiki_json = json.load(f)
df_actor_wiki = pd.read_excel("../data/wiki_raw.xlsx" )



In [ ]:
# df_new8years["time_table"]
df_new8years.columns

# hoge



In [ ]:
df_actor_wiki[df_actor_wiki["office"] == "ジャニーズ事務所[4]"]["office"] = "ジャニーズ事務所"

In [ ]:
target_idx = df_actor_wiki[df_actor_wiki["office"] == "ジャニーズ事務所[4]"].index.values

In [ ]:
df_actor_wiki["office"].iloc[target_idx] = "ジャニーズ事務所"

In [ ]:
df_actor_wiki.iloc[target_idx]["office"]

In [ ]:
# Jかどうか
df_actor_wiki["isj"] = df_actor_wiki["office"] == "ジャニーズ事務所"

In [ ]:
# df_new8years["actor1_office"] = np.nan
# df_new8years["actor2_office"] = np.nan
# df_new8years["actor1_birthday"] = np.nan
# df_new8years["actor2_birthday"] = np.nan
df_new8years["actor1_isj"] = 0
df_new8years["actor2_isj"] = 0

In [ ]:
for idx in range(len(df_new8years)):
    actor1 = df_new8years["main_actor1"].iloc[idx]
    actor2 = df_new8years["main_actor2"].iloc[idx]
    
    if actor1 in  df_actor_wiki["actor_name"].values :
        df_new8years["actor1_isj"].iloc[idx] = \
            int(df_actor_wiki[df_actor_wiki["actor_name"] == actor1].isj.values[0])
    
    if actor2 in  df_actor_wiki["actor_name"].values :
        df_new8years["actor2_isj"].iloc[idx] = \
            int(df_actor_wiki[df_actor_wiki["actor_name"] == actor2].isj.values[0])
        

In [ ]:
# df_new8years[df_new8years["actor1_isj"]==True]

In [ ]:
# for idx in range(len(df_new8years)):
#     actor1 = df_new8years["main_actor1"].iloc[idx]
#     actor2 = df_new8years["main_actor2"].iloc[idx]
# #     print(actor1)
    
#     if actor_wiki_json[actor1]:
    
#         df_new8years["actor1_office"].iloc[idx] = actor_wiki_json[actor1].get("office")
#         df_new8years["actor1_birthday"].iloc[idx] = actor_wiki_json[actor1].get("birthday")
    
#     if actor_wiki_json[actor2]:
#         df_new8years["actor2_office"].iloc[idx] = actor_wiki_json[actor2].get("office")
#         df_new8years["actor2_birthday"].iloc[idx] = actor_wiki_json[actor2].get("birthday")

### dummy化

In [ ]:
X = df_new8years[["main_actor1_pt" , "main_actor2_pt", "actor1_isj","actor2_isj", \
                  "pre_part","pre_slot_vr_average","has_original_work",\
                  "is_than_part3",]]
#                   "year_feature"]]

In [ ]:
# X_dummys = df_new8years.loc[:,df_new8years.columns.str.contains(('start_at_year_|TV_station_|start_at_month_|time_table'))]
X_dummys = df_new8years.loc[:,df_new8years.columns.str.contains(('start_at_year_|start_at_month_|time_table'))]

In [ ]:
X_dummys

In [ ]:
X = pd.concat([X,X_dummys], axis=1)


#### モデル作成

In [ ]:
y = df_new8years["average"]

In [ ]:
lm = LinearRegression()

In [ ]:
# X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=1234 )
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=12345 )

In [ ]:
model=lm.fit(X_train,y_train)
#predictions=model.predict(x_test)

In [ ]:
y_pred = lm.predict(X_test)

In [ ]:
lm_rmse = np.sqrt(mean_squared_error(y_test, y_pred))

In [ ]:
print(lm_rmse)

In [ ]:
for i in range(len(y_pred)):
    a = y_test.values[i]
    b = y_pred[i]
    c = df_new8years.iloc[y_test.index[i]].drama_title
    print(f'{c}\t 視聴率：実際 {a}   \t 予測 {b} ')

In [ ]:
for i in range(len(y_pred)):
    a = y_test.values[i]
    b = y_pred[i]
    c = df_new8years.iloc[y_test.index[i]].drama_title
    d = df_new8years.iloc[y_test.index[i]].main_actor1_pt
    e = df_new8years.iloc[y_test.index[i]].main_actor2_pt
    f = df_new8years.iloc[y_test.index[i]].pre_part
    g = df_new8years.iloc[y_test.index[i]].pre_slot_vr_average
    if abs (a-b) > 2:
        print(f'{c}\t 視聴率：実際 {a}   \t 予測 {b} ,{d} ,{e},{f}, {g}')

In [ ]:
for i in range (len (X.columns)):
    print(f"""{i}  回帰係数 {X.columns[i]}\t{model.coef_[i]} """)

In [ ]:
print('モデル関数の切片 w2: %.3f' %model.intercept_)

In [ ]:
print('決定係数 R^2： ', model.score(X, y))

In [ ]:
# lm_rmse = np.sqrt(mean_squared_error(y_test, y_pred))
# print (f"rmse: {lm_rmse}")

# モデル２決定木

In [ ]:
# clf = DecisionTreeRegressor(max_depth = 3)
clf = DecisionTreeRegressor(max_leaf_nodes= 16)
# clf = DecisionTreeRegressor(max_leaf_nodes= 12)
clf.fit(X_train, y_train)

In [ ]:
y_pred_tree = clf.predict(X_test)

In [ ]:
rf_rmse = np.sqrt(mean_squared_error(y_test, y_pred_tree))
print (rf_rmse)

In [ ]:
for i in range (len (X.columns)):
    print(f"""{i}  FI {X.columns[i]}     \t{clf.feature_importances_[i]} """)


In [ ]:
for i in range(len(y_pred)):
    a = y_test.values[i]
    b = y_pred[i]
    print(f'視聴率：実際 {a}   \t 予測 {b} ')

In [ ]:
dot_data = tree.export_graphviz(
    clf,
    out_file=None,
    feature_names=X.columns,
    class_names="average",
    filled=True,
    proportion=True)
graph = pydotplus.graph_from_dot_data(dot_data)
Image(graph.create_png())

### モデル３　線形回帰 目的変数を初回に

In [ ]:
y1 = df_new8years["Episode1"]

In [ ]:
lm1 = LinearRegression()

In [ ]:
# X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=1234 )
X_train1,X_test1,y_train1,y_test1=train_test_split(X,y1,test_size=0.2,random_state=12345 )

In [ ]:
model1 = lm1.fit(X_train1,y_train1)
#predictions=model.predict(x_test)

In [ ]:
y_pred1 = lm1.predict(X_test1)

In [ ]:
lm1_rmse = np.sqrt(mean_squared_error(y_test1, y_pred1))

In [ ]:
print(lm1_rmse)

In [ ]:
for i in range(len(y_pred1)):
    a = y_test1.values[i]
    b = y_pred1[i]
    c = df_new8years.iloc[y_test1.index[i]].drama_title
    print(f'{c}\t 視聴率：実際 {a}   \t 予測 {b} ')

In [ ]:
for i in range(len(y_pred1)):
    a = y_test1.values[i]
    b = y_pred1[i]
    c = df_new8years.iloc[y_test1.index[i]].drama_title
    if abs (a-b) > 3:
        print(f'{c}\t 視聴率：実際 {a}   \t 予測 {b} ')

In [ ]:
for i in range (len (X.columns)):
    print(f"""{i}  回帰係数 {X.columns[i]}\t{model1.coef_[i]} """)

In [ ]:
print('モデル関数の切片 w2: %.3f' %model1.intercept_)

In [ ]:
print('決定係数 R^2： ', model1.score(X, y1))

In [ ]:
from __future__ import print_function
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
from statsmodels.sandbox.regression.predstd import wls_prediction_std

np.random.seed(9876789)

In [ ]:
# Fit and summary:

# Fit and summary:

model_sm = sm.OLS(y, sm.add_constant(X))
res = model_sm.fit()
print(res.summary())

In [ ]:
len(res.predict())

In [ ]:
res.predict()[y_test.index]

In [ ]:
def rmse_accuracy_percentage(actual,predicted): 
    a = actual
    b = predicted
    print("RMSE is:",np.round(np.sqrt(sum(((np.array(a)-np.array(b))**2))/len(a)),2))

In [ ]:
rmse_accuracy_percentage( y_test, res.predict()[y_test.index])

In [ ]:
model_sm1 = sm.OLS(y1, sm.add_constant(X))
res1 = model_sm1.fit()
print(res1.summary())

In [ ]:
rmse_accuracy_percentage( y_test, res1.predict()[y_test.index])